In [1]:
# Execute with lates qiskit to keep the time CX comparisons equal.

import time
from qiskit import transpile
import json
import os
import pickle
from qiskit import QuantumCircuit
from copy import deepcopy
# current_dir=os.getcwd()
# os.chdir("../Paulihedral_v2/Paulihedral_new")
# from benchmark.mypauli import pauliString
# import synthesis_FT, parallel_bl
# import importlib
# importlib.reload(synthesis_FT)
# importlib.reload(parallel_bl)
# os.chdir(current_dir)

In [2]:
#Load the pickle files.
def run_experiment_folder(folder_path = None, filename = None, save_output = False, names_start_with=""):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".pkl") and filename.startswith(names_start_with):
            # Print the filename
            print(filename)
            with open(folder_path + filename, "rb") as file:
                data = pickle.load(file)
            # print(data)
            paulis=data["paulis"]
            circuit=QuantumCircuit.from_qasm_str(data["circuit"])
            results=data["results"][0]
            tot_time=results["times"]["paulihedral_time"]
            number_of_ham=results["num_paulis"]
            # Measure time for method
            start_time = time.time()
            circuit = transpile(circuit, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
            end_time = time.time()
            # include the transpilation time.
            tot_time+= (end_time - start_time)
            results = []
            # Collect results
            new_filename=filename[0:-len(".pkl")]+".json"

            if filename.startswith(("max_cut", "labs")): # we will multiply the number of cnot by the layer count since
                # ph does not optimize across qaoa layers.
                layers=int(len(paulis)/2)
            else:
                layers=1
            print("found layers: ", layers)
            # print("original ", circuit)
            circuit=circuit.repeat(layers)
            circuit=circuit.decompose()
            # print("repeated ", circuit)
            result = {
                "num_paulis": number_of_ham,
                "times": {
                    "paulihedral_time": tot_time
                },
                "gate_counts": {
                    "paulihedral_method": circuit.count_ops().get('cx', 0)*layers
                },
                "circuit_entangling_depth": {
                    "paulihedral_method": circuit.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results/test_paulihedral_' + new_filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_paulihedral_' + new_filename, 'w') as paulis_file:
                    json.dump([paulis, results], paulis_file, indent=4)
    

    

In [3]:
run_experiment_folder(folder_path="benchmarks/paulihedral_partial_results/", save_output=True, names_start_with=("max_cut", "labs"))

labs_n10_layers1.pkl
found layers:  1
{'num_paulis': 70, 'times': {'paulihedral_time': 1.9484312534332275}, 'gate_counts': {'paulihedral_method': 230}, 'circuit_entangling_depth': {'paulihedral_method': 209}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_labs_n10_layers1.json'}
labs_n10_layers3.pkl
found layers:  3
{'num_paulis': 70, 'times': {'paulihedral_time': 0.0997304916381836}, 'gate_counts': {'paulihedral_method': 2070}, 'circuit_entangling_depth': {'paulihedral_method': 605}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_labs_n10_layers3.json'}
labs_n11_layers1.pkl
found layers:  1
{'num_paulis': 95, 'times': {'paulihedral_time': 0.15520143508911133}, 'gate_counts': {'paulihedral_method': 316}, 'circuit_entangling_depth': {'paulihedral_method': 287}, 'test_paulis_file': 'benchmarks/results/test_paulihedral_labs_n11_layers1.json'}
labs_n11_layers3.pkl
found layers:  3
{'num_paulis': 95, 'times': {'paulihedral_time': 0.16338491439819336}, 'gate_counts': {'pa

In [4]:
import qiskit
print(qiskit.__version__)

1.1.1
